## Lab 13: Path Planning and Execution (real)

### Plan



**Goal:** Navigate through a series of waypoints as quickly and accurately as possible.

**Process:**
1. Use the Bayes filter to establish a belief about the robot's initial position
2. Find the next waypoint in the list of waypoints to navigate
3. Calculate the control (change in angle, change in distance) needed to get to the waypoint
4. Use turn_degrees() to turn
5. Move forward with ToF-based PID (and ToF averaging because of all the noise)
6. Spin around and localize again to make sure you're on track
7. Repeat until you've seen all the waypoints

### Setup

In [2]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

START_PLOTTER()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-05-14 14:41:47,769 | INFO     |: Logger demo_notebook.log already initialized.
2022-05-14 14:41:47,771 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.
2022-05-14 14:41:47,772 | ERROR    |: Plotter is already running


### Set up the robot 

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        # Data collection stuff
        self.dist_data = []
        self.angle_data = []
        
        # Current pose tracking
        self.current_pose = (0,0,0)

        self.setup_notify()
        

    def setup_notify(self):
        # print("Notifications started")
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        # print(ble.uuid['RX_TOF1'])
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        # print(f"Got TOF data: {new_tof}")
        self.dist_data.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        # print(f"Got angle data: {new_angle}")
        self.angle_data.append( new_angle )
        
    def set_params(self, pid_p_rot, pid_min_power_rot, pid_p_fwd, pid_min_power_fwd, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p_rot}|{pid_min_power_rot}|{pid_p_fwd}|{pid_min_power_fwd}|{imu_hz}"); 

    def set_motor_calib(self, calib_rot, calib_fwd):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, f"{calib_rot}|{calib_fwd}")
        
    def move_distance(self, dist_mm):
        self.ble.send_command(CMD.MOVE_DISTANCE, dist_mm)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)
        
    def move_duration(self, power, duration, pause=False):
        self.ble.send_command(CMD.MOVE_DURATION, f"{power}|{duration}")
        time.sleep(duration / 1000)
        if pause:
            robot.move_duration(0,250, pause=False)
            

    def dist_to_time(self, d):
        # distance (cm) to time (ms)
        # return round(69.04669* (d**0.644513))
        return round(60 * (d**0.644513))
    
    def move_dist_as_duration(self, dist, pause=False):
        t = self.dist_to_time(dist / 10)
        self.move_duration(75, t, pause=pause)
        
    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        return self.current_pose

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # Clear old data if it exists
        self.dist_data = []
        self.angle_data = []
        self.ble.send_command(CMD.OBSERVE, None)
        while len(self.dist_data) < 18 or len(self.angle_data) < 18:
            # print("Data still sending, waiting")
            self.dist_data = []
            self.angle_data = []
            time.sleep(1)
            self.ble.send_command(CMD.GET_DATA, None) # try again for command not sent?
        self.dist_data = [x / 1000 for x in self.dist_data]
        temp_angles = []
        temp_sum = 0
        for i,angle in enumerate(self.angle_data):
            temp_sum -= angle
            temp_angles.append(temp_sum)
        self.angle_data = temp_angles
        print("Observation loop complete")
        print(f"Distance data: {self.dist_data}")
        print(f"Angle data: {self.angle_data}")
        print("")
        return np.array(self.dist_data).reshape((18,1)), np.array(self.angle_data).reshape((18,1))

def ft2meters(pt):
    meters_in_one_foot = 0.3048
    return (pt[0] * meters_in_one_foot, pt[1] * meters_in_one_foot)



### Connect via Bluetooth

In [17]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

time.sleep(3)

# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)
print("Ready")


2022-05-14 14:50:33,087 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:07:21:8D:B3:44
2022-05-14 14:50:37,558 | INFO     |: Connected to C0:07:21:8D:B3:44
Ready


In [23]:
"""
Test stuff
"""
print("beginning test")
# Params: PID_P_ROT, PID_MIN_ROT, PID_P_FWD, PID_MIN_FWD, IMU_HZ
robot.set_params(4, 100, 0.5, 35, 60) # Old: 1.4 / 80 / 0.5 / 35 / 60

# Params: Rotational, then Forward
robot.set_motor_calib(1.3, 1.5) # Old: 1.5, 1

robot.turn_degrees(-90)
# robot.move_distance(300)
# for _ in range(4):
#     robot.move_dist_as_duration(300, pause=True)
#     robot.turn_degrees(90)
#     robot.move_duration(0,250)
#     print("ding")

print("Test done")

beginning test


Exception: Not connected to a BLE device

### Run the Bayes Filter for inital localization

In [14]:
# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

# Reset Robot and Plots
# robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
# then update
# loc.get_observation_data()

# loc.update_step()
# loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom = robot.get_pose()
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-14 14:47:40,674 | INFO     |:  | Number of observations per grid cell: 18
2022-05-14 14:47:40,675 | INFO     |:  | Precaching Views...


/Users/main/Documents/Classes/2022-sp/ECE-4960/ece4960-fast-robots-code/lab10/ECE4960-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-14 14:47:44,649 | INFO     |:  | Precaching Time: 3.973 secs
2022-05-14 14:47:44,650 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-14 14:47:44,651 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-05-14 14:47:44,859 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-14 14:47:44,860 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


### Follow the trajectory

In [15]:
start = (-4,-3)

points = [
    (-2, -1),
    (1, -1),
    # (2, -3),
    # (5, -3),
    # (5, -2),
    # (5, 3),
    # (0, 3),
    # (0, 0)
]
# Params: PID_P_ROT, PID_MIN_ROT, PID_P_FWD, PID_MIN_FWD, IMU_HZ
robot.set_params(1.7, 80, 0.5, 35, 60)
robot.set_motor_calib(1.3, 1.5)

# start = (0,0)

# points = [
#     (1,0), (1,1), (0,1), (0,0)
# ]

# Assume you perfectly go EVERYWHERE, so you can pre-compute all controls
start = ft2meters(start)
waypoints = [ft2meters(pt) for pt in points]

current_loc = (start[0], start[1], 0)
for i, w in enumerate(waypoints):
    # TODO navigate from current location to waypoint
    print(f"{current_loc} -> {w}")
    sx, sy, sa = current_loc
    dx, dy = w
    
    da = 0
    # TODO compute the angle for the next waypoint, fix calculations
    if i+1 < len(waypoints):
        (x1,y1)= (np.cos(np.radians(sa)), np.sin(np.radians(sa)))
        (x2,y2) = waypoints[i+1]
        x2 = x2 - dx
        y2 = y2 - dy
        # print(f"Vector 1: {x1}, {y1}")
        # print(f"Vector 2: {x2}, {y2}")
        denom = np.sqrt(x1**2 + y1**2) * np.sqrt(x2**2 + y2**2)
        numer = x1*x2 + y1*y2
        # print(denom)
        # print(numer)
        da = np.degrees(np.arccos(numer/denom))
    
    print(f"Computing control: {(sx,sy,sa)} -> {(dx,dy,da)}")
    (a1,x1,a2) = loc.compute_control((dx,dy,da), (sx,sy,sa))
    
    print(f"Control: {(a1,x1,a2)}")
    print()
    
    robot.turn_degrees(-1*a1)
    robot.move_duration(0,250)
    robot.move_distance(x1*1000)
    # robot.move_dist_as_duration(x1*1000, pause=True)
    robot.turn_degrees(-1*a2)
    robot.move_duration(0,250)
    
    current_loc = (dx,dy,da)
  

(-1.2192, -0.9144000000000001, 0) -> (-0.6096, -0.3048)
Computing control: (-1.2192, -0.9144000000000001, 0) -> (-0.6096, -0.3048, 0.0)
Control: (45.00000000000001, 0.8621045876226389, -45.00000000000001)

(-0.6096, -0.3048, 0.0) -> (0.3048, -0.3048)
Computing control: (-0.6096, -0.3048, 0.0) -> (0.3048, -0.3048, 0)
Control: (0.0, 0.9144000000000001, 0.0)

2022-05-14 14:49:14,128 | INFO     |: Disconnected from 7AE0A3CC-63D5-13D9-39BA-D29A7DC67D14


BleakError: disconnected